# Misc tests

In [ ]:
from dolfin import *
# create mesh and define function space
mesh = UnitCubeMesh(32, 32, 32)
V = FunctionSpace(mesh, "Lagrange", 1)
# define Dirichlet Boundary at bottom face
def boundary(x):
    return x[0] < DOLFIN_EPS

# define boundary condition
bc = DirichletBC(V, Constant(0.0), boundary)
# define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Expression("10 * exp(-(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2) + pow(x[2] - 0.5, 2)) / 0.02)", degree=1)
a = inner(grad(u), grad(v)) * dx
L = f*v*dx

# Compute solution
u = Function(V)
solve(a == L, u, bc)

In [ ]:
plot(u, cmap='bone')

In [ ]:
print("This sometimes screws the notebook...")
#from IPython.core.display import HTML

#HTML(X3DOM.html(u))

# Periodic boundary conditions

In [ ]:
from dolfin import *
from ffc.log import add_logfile, set_level, DEBUG
set_level(DEBUG)
add_logfile("/tmp/fenics.log")

class PeriodicBoundary(SubDomain):

    def inside(self, x, on_boundary):
        return bool((near(x[0], 0) or near(x[1], 0)) and 
                (not ((near(x[0], 0) and near(x[1], 1)) or 
                        (near(x[0], 1) and near(x[1], 0)))) and on_boundary)

    def map(self, x, y):
        if near(x[0], 1) and near(x[1], 1):
            y[0] = x[0] - 1.
            y[1] = x[1] - 1.
        elif near(x[0], 1):
            y[0] = x[0] - 1.
            y[1] = x[1]
        else:  
            y[0] = x[0]
            y[1] = x[1] - 1.

k = 5
n = 2**k 
mesh = UnitSquareMesh(n,n)
V = VectorFunctionSpace(mesh, "CG", 1, constrained_domain = PeriodicBoundary())

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)
s = Constant("1.0")
uex = Expression(("-cos(2*DOLFIN_PI*x[1])","-sin(2*DOLFIN_PI*x[0])"), element=V.ufl_element())
t = Expression(("4*DOLFIN_PI*DOLFIN_PI*cos(2*DOLFIN_PI*x[1])","4*DOLFIN_PI*DOLFIN_PI*sin(2*DOLFIN_PI*x[0])"), element = V.ufl_element())
a = -s*inner(grad(u), grad(v))*dx
L = +s*div(v)*dx + inner(t, v)*dx

u = Function(V)

In [ ]:
problem = LinearVariationalProblem(a, L, u, bcs=[])

In [ ]:
solver = LinearVariationalSolver(problem) 
solver.parameters["linear_solver"] = "gmres"
solver.parameters["preconditioner"] = "hypre_amg"
solver.parameters["krylov_solver"]["monitor_convergence"] = False
solver.parameters["krylov_solver"]["relative_tolerance"] = 1e-10

In [ ]:
solver.solve()

In [ ]:
file = File("periodic_trial_%i.pvd" % k)
file << u
file2 = File("periodic_exact.pvd")
u_exact =interpolate(uex, V) 
file2 << u_exact
print("error %f" % assemble(inner(u-u_exact, u-u_exact) * dx))